<a id='home_cell'></a>
## BLOG POST - Moderating Variables 

*Please note I have chosen to use GitHub / Jupyter notebooks in substitute of Tumblr for my blog assignments

I have chosen to analyze the correlations of my prior continuous variables, but this time with a split out of the
"purpose" variable.  This "purpose" variable is a high level explanative reason for the request of the loan. 
It is also the highest level of a hiearchy breakout of the data.  The other variables include "interest rate", "revolving utilization rate", "debt to income ratio", and 
adding "annual income" for good measure.  

Upon analyzing the variables with the purpose of the loan, the correlation analysis doesn't look very differnt from
prior analysis.  Previously I had identified a mild correlation at most between all the variables.
When I dive into the defaulted breakout data set, I can start to see some interesting trends.
When the purpose of the loan is "renewable_energy" - you can see a very high correlation between the revolving utilization and 
the interest rate. Also the "vacation" category has correlations that peaks past the .50 mark to .51 .  The only drawback
to this conclusion is the sample size of these groups which is extremely small (20 and 59 samples).  This may not be a large
enough sample of a population, or it may be acceptable because the company does not receive that many requests.  A 
follow up with a business anlayst might help shed more light.

So given these two sub-sets of the purpose of the loan - I can expect that a higher interest rate is correlated with a higher revolving 
utilization rate within certain categories.  I could start to speculate that in those two categories, anyone seeking loans with a high
debt-to-income ratio might be riskier and therefore recieve a higher interest rate.   

In [1]:
# imports
import pandas
import numpy
import seaborn
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 

import scipy.stats

# for display multiple data frames below
from IPython.core import display as ICD

In [2]:
# =====================
# 2. DATA IMPORT 
# =====================
# set working directory
directory = '//Project/OneDrive/Coursera/Wesleyan_Course1/wk1_Assignment/'

pdLoans = pandas.read_csv(directory + 'LoanStats3a.csv', low_memory=False)

# =======================
#  3. DATA TRANSFORMATIONS 
#  ======================= 
# Convert the data elements - just doing in 
pdLoans["int_rate_d"] = pandas.to_numeric(pdLoans["int_rate"].str.replace('%',''), errors='coerce')
pdLoans["revol_util_d"] = pandas.to_numeric(pdLoans["revol_util"].str.replace('%',''), errors='coerce')

# divide by 100 to get the true percentage
pdLoans["int_rate_d"] = pdLoans["int_rate_d"] / 100.00
pdLoans["revol_util_d"] = pdLoans["revol_util_d"] / 100.00

# Convert and Bin the data
pdLoans['int_rate_bin'] = pandas.cut(pdLoans.int_rate_d, [0.0,0.10,0.15,0.20,0.25,0.30])
pdLoans['dti_bin'] = pandas.cut(pdLoans.dti, [0,1,10,20,30])
pdLoans['revol_util_bin'] = pandas.cut(pdLoans.revol_util_d, [0,0.25,0.5,0.75,1])

# Add a veriable for recoveries / collection_recovery_fee.  
# If either is greather than 0, then assign a 1 to indicate a default
# Create a function to Map
def set_Defaulted (row):
    if row['recoveries'] > 0:
        return 1
    if row['collection_recovery_fee'] > 0:
        return 1
    else:
        return 0 

pdLoans['defaulted'] = pdLoans.apply(lambda row: set_Defaulted(row), axis=1)

# Here split out separate date set for the defaulted loans - for analayis use later
pdLoansDefaulted = pdLoans[pdLoans['defaulted'] == 1]

In [3]:
## Potential Moderator Variables 

In [4]:
# Clean out any na's as I will be using the correlation for the variables (grouped by categorical)
pdLoans = pdLoans.dropna()
pdLoansDefaulted = pdLoansDefaulted.dropna()

In [5]:
# Subset out to only the variables I am interested in analyzing
# will use this to do matrix correlations with
pdLoansSub = pdLoans[['purpose','int_rate_d','dti','revol_util_d','annual_inc']]
pdLoansSub.head()

pdLoansDefaultedSub = pdLoansDefaulted[['purpose','int_rate_d','dti','revol_util_d','annual_inc']]
pdLoansDefaultedSub.head()

,purpose,int_rate_d,dti,revol_util_d,annual_inc
27,other,0.1677,13.97,0.595,50004.0
71,debt_consolidation,0.1677,20.22,0.675,75000.0
114,credit_card,0.1825,24.14,0.963,75000.0
120,credit_card,0.1349,14.15,0.912,67000.0
121,debt_consolidation,0.1065,22.80,0.542,45000.0


In [6]:
pdLoans.purpose.unique()

array(['other', 'home_improvement', 'medical', 'debt_consolidation',
       'small_business', 'credit_card', 'car', 'major_purchase', 'house',
       'vacation', 'wedding', 'moving', 'renewable_energy', 'educational'], dtype=object)

In [7]:
# Assign the purpose of the loan by grabbing the unique values in the "purpose" column.
# Then loop through each "purpose" of the loan and perform a correlation matrix on each of the variables
purposeList = pdLoansSub.purpose.unique()
purposeList.sort()

for purposeName in purposeList:
    print(purposeName)
    corr = pdLoansSub[pdLoansSub['purpose']==purposeName].corr()
    ICD.display(corr)
    print('')

car


,int_rate_d,dti,revol_util_d,annual_inc
int_rate_d,1.000000,0.040478,0.367449,-0.064433
dti,0.040478,1.000000,0.225557,-0.057979
revol_util_d,0.367449,0.225557,1.000000,0.022098
annual_inc,-0.064433,-0.057979,0.022098,1.000000



credit_card


,int_rate_d,dti,revol_util_d,annual_inc
int_rate_d,1.000000,0.150737,0.478638,0.126150
dti,0.150737,1.000000,0.182148,-0.177029
revol_util_d,0.478638,0.182148,1.000000,0.046879
annual_inc,0.126150,-0.177029,0.046879,1.000000



debt_consolidation


...

,int_rate_d,dti,revol_util_d,annual_inc
int_rate_d,1.000000,0.069677,0.367371,0.085735
dti,0.069677,1.000000,0.199043,-0.079972
revol_util_d,0.367371,0.199043,1.000000,0.055061
annual_inc,0.085735,-0.079972,0.055061,1.000000



vacation


,int_rate_d,dti,revol_util_d,annual_inc
int_rate_d,1.000000,0.137222,0.448500,0.121935
dti,0.137222,1.000000,0.295982,-0.104262
revol_util_d,0.448500,0.295982,1.000000,0.143747
annual_inc,0.121935,-0.104262,0.143747,1.000000



wedding


,int_rate_d,dti,revol_util_d,annual_inc
int_rate_d,1.000000,0.064806,0.285886,0.093968
dti,0.064806,1.000000,0.187796,-0.138328
revol_util_d,0.285886,0.187796,1.000000,-0.006381
annual_inc,0.093968,-0.138328,-0.006381,1.000000


In [8]:
# Assign the purpose of the loan by grabbing the unique values in the "purpose" column
purposeList = pdLoansDefaultedSub.purpose.unique()
purposeList.sort()

for purposeName in purposeList:
    print(purposeName)
    corr = pdLoansDefaultedSub[pdLoansDefaultedSub['purpose']==purposeName].corr()
    ICD.display(corr)
    print('')
    

    

car


,int_rate_d,dti,revol_util_d,annual_inc
int_rate_d,1.000000,-0.132533,0.278504,-0.010476
dti,-0.132533,1.000000,0.011623,-0.156273
revol_util_d,0.278504,0.011623,1.000000,0.028910
annual_inc,-0.010476,-0.156273,0.028910,1.000000



credit_card


,int_rate_d,dti,revol_util_d,annual_inc
int_rate_d,1.000000,0.165019,0.381938,0.137221
dti,0.165019,1.000000,0.201600,-0.266684
revol_util_d,0.381938,0.201600,1.000000,-0.009108
annual_inc,0.137221,-0.266684,-0.009108,1.000000



debt_consolidation


...

,int_rate_d,dti,revol_util_d,annual_inc
int_rate_d,1.000000,0.050533,0.270306,-0.003409
dti,0.050533,1.000000,0.151801,-0.071210
revol_util_d,0.270306,0.151801,1.000000,-0.053548
annual_inc,-0.003409,-0.071210,-0.053548,1.000000



vacation


,int_rate_d,dti,revol_util_d,annual_inc
int_rate_d,1.000000,0.326978,0.510106,0.106810
dti,0.326978,1.000000,0.111048,0.174243
revol_util_d,0.510106,0.111048,1.000000,0.145953
annual_inc,0.106810,0.174243,0.145953,1.000000



wedding


,int_rate_d,dti,revol_util_d,annual_inc
int_rate_d,1.000000,-0.035532,0.134996,0.392293
dti,-0.035532,1.000000,0.083845,0.042752
revol_util_d,0.134996,0.083845,1.000000,0.247352
annual_inc,0.392293,0.042752,0.247352,1.000000
